In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

import re

In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
##############
# Parameters #
##############

min_gram = 1
max_gram = 3

# To create ours partitions, we must first know the years which will be the limits
limit_years = [2007, 2010, 2013, 2016]

# Ignore words that appear at a frequency less than max_frequ in the corpus
max_frequ = 0.8

# Ignore words appearing less than min_appear in the whole corpus
min_appear = 5

# Range fo cluster number you want to test
cluster_ranges = range(2, 30)

# Number of trial you want to do for each test
nb_trial_by_test = 3

In [4]:
# Datas preprocessing methods.

# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw
sourceFST += [x.replace('\n', '') for x in open('perso_words-fr.txt', mode="r", encoding="utf-8").readlines()]

# Based on ration of french and english stopwords
def isEnglish(article):
    total_fsw = len([x for x in article.split() if x in sourceFST])
    total_esw = len([x for x in article.split() if x in stopwords.words('english')])
    ratio = 100
    if total_fsw != 0:
        ratio = total_esw/total_fsw
    return ratio > 1 and total_esw > 3

def lemmatize(article):
    arti_lower = article.lower()
    arti_2words = re.sub(" [0-z][0-z] ", " ", arti_lower) # word of length < 2
    arti_e = re.sub("(é|è|ê)", "e", arti_2words)
    arti_o = re.sub("à", "a", arti_e)
    arti_i = re.sub("ô", "o", arti_o)
    artiregex = re.sub("î", "i", arti_i)
    output = []
    outPonc = artiregex.translate(artiregex.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST and [x for x in token.lemma_ if x not in "0123456789"] != []:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [5]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [6]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    #if not null, empty, or whatever (so if there is a abstract):
    if not isinstance(data['abstract'][i], float) and not isEnglish(data['abstract'][i]):
        text = data['abstract'][i]
        if not isinstance(data['title'][i], float):
            text += " "+data['title'][i]

        numArti+=1
        usable.append(re.sub(" [0-z][0-z] ", " ", stemmer.stem(lemmatize(text))))
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti)
limits.append(numArti)



In [7]:
# Post-process word removal
post_words = [x.replace('\n', '') for x in open('post_process_words-fr.txt', mode="r", encoding="utf-8").readlines()]

for i in range(0, len(usable)):
    arti = usable[i].split()
    res = []
    for word in arti:
        if word not in post_words:
            res.append(word)
    usable[i] = ' '.join(res)

In [8]:
# Display pre-processed datas

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram), max_df=max_frequ, min_df=min_appear)
tfidf = vectorizer.fit_transform(usable)

print("nombre d'articles =", len(usable))
print("nombre de mots =", len(tfidf.toarray()[0]))
print("limits =", limits)

usable[0]

/info/etu/m2/i140302/venv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['quelqu'] not in stop_words.
  sorted(inconsistent))


nombre d'articles = 991
nombre de mots = 2353
limits = [114, 301, 468, 694, 991]


'plateforme objectif permettre citoyen euxmemer tweet politique devenement specifiqu francepour cas lelection presidentiell ideo2017 analyser quasitemps reel message candidat fournir principal caracteristiqueslusage lexiqu politique comparaison entrer candidat ideo2017 plateforme citoyen dediee lanalyse tweet evenement polit'

In [9]:
# Creation of partitions_tfidf[], which give us the TFIDF of each cluster of each partition
# partitions_tfidf[num_partition][num_doc][num_word]
# Beware, num_doc can't be equals to 1091 (max). You have partitions, so every doc aren't in every partitions
# num_word can be found via vectorizer.get_feature_name()
partitions_tfidf = []
beg = 0
for l in limits:
    last = l
    partitions_tfidf.append([list(x) for x in list(tfidf.toarray())[beg:last]])
    beg = l

In [10]:
vectorizer.get_feature_names()

['acce',
 'accessible',
 'achat',
 'acquisition',
 'acquérir',
 'acteur',
 'actif',
 'action',
 'activite',
 'actuel',
 'actuellement',
 'adapt',
 'adaptatif',
 'adaptation',
 'adapte',
 'adaptee',
 'adapter',
 'adequat',
 'adn',
 'adopter',
 'afc',
 'affiner',
 'agent',
 'agregation',
 'aid',
 'aider',
 'ainsiqu',
 'ajoutee',
 'ajouter',
 'ala',
 'aleatoir',
 'algebriqu',
 'algorithm',
 'algorithm classification',
 'algorithm dapprentissage',
 'algorithm dextraction',
 'algorithm efficace',
 'algorithme',
 'algorithme dapprentissage',
 'algorithme dextraction',
 'algorithme fouiller',
 'algorithme incremental',
 'algorithmique',
 'alignement',
 'alternatif',
 'amelior',
 'amelioration',
 'ameliore',
 'amelioree',
 'ameliorer',
 'ameliorer qualite',
 'amene',
 'amont',
 'analys',
 'analyse',
 'analyser',
 'analyser donnee',
 'analyser factoriel',
 'analyser semantiqu',
 'analytique',
 'anne',
 'annees',
 'annot',
 'annotation',
 'annotation semantiqu',
 'annoter',
 'anormal',
 'anr',
 

# KMeans & Silhouette Score

In [11]:
# Applying KMeans on tfidf
# the labels_ give assignment of doc to the cluster number 


In [12]:
# doc_clustering is a dictionnary 
# it looks like -> { doc_number : [partition_number, cluster_number] }
# This is used to reassign doc number to their respective partition and and cluster

def kmeans(nb_clusters):
    doc_clustering = {}
    
    km = KMeans(n_clusters=nb_clusters)

    # Silhouette score mean
    silhouette_mean = 0

    numDoc = 0
    for i in range(0, len(limits)):
        dash = km.fit(partitions_tfidf[i])

        # Silhouette
        silhouette_mean += silhouette_score(partitions_tfidf[i], dash.labels_)

        previousBound = 0
        if i > 0:
            previousBound = limits[i-1]
        for numDocItern in range(0, limits[i]-previousBound):
            doc_clustering[numDoc] = [i, dash.labels_[numDocItern]]
            numDoc+=1

    silhouette_mean = silhouette_mean / len(limits)
    res = {}
    res["silhouette"] = silhouette_mean
    res["clustering"] = doc_clustering
    return res


In [13]:
# Compute Silhouette Score for each number of cluster

silhouette_by_cluster_nb = {}

for nb_clusters in cluster_ranges:
    silhouette_avg = 0
    for trial in range(0, nb_trial_by_test):
        km = kmeans(nb_clusters)
        silhouette_avg += km["silhouette"]
    silhouette_avg = silhouette_avg / nb_trial_by_test
    silhouette_by_cluster_nb[nb_clusters] = silhouette_avg

In [14]:
# We want silhouette scores to be high
silhouette_by_cluster_nb

{2: 0.00578205545072553,
 3: 0.005859485301478853,
 4: 0.006947141674082224,
 5: 0.0076449044031812975,
 6: 0.00863541884545567,
 7: 0.009097191680325507,
 8: 0.010283301950888975,
 9: 0.011125581347600952,
 10: 0.012199983527272367,
 11: 0.013407125870923258,
 12: 0.013401377450969637,
 13: 0.013806295770944067,
 14: 0.014532242413086002,
 15: 0.015638030584180243,
 16: 0.016143565582583584,
 17: 0.017349385658343926,
 18: 0.017327238367165192,
 19: 0.018444712635895956,
 20: 0.018258440418140218,
 21: 0.020663557779260495,
 22: 0.02085535316646132,
 23: 0.020957775875229307,
 24: 0.021521382784714438,
 25: 0.022189173848375404,
 26: 0.02322660672229924,
 27: 0.023700977824183723,
 28: 0.022771922014560894,
 29: 0.024773691200101925}

In [15]:
doc_clustering = kmeans(5)["clustering"]

In [16]:
# Allows to get list of documents number
# return [dou numbers]
# params : partition_number , cluster number
def get_doc(part, clust):
    docs = []
    for i in range(0,len(doc_clustering)):
        if doc_clustering[i][0] == part and doc_clustering[i][1] == clust:
            docs.append(i)
    return docs

In [17]:
# Get the partitions variable
# Here partitions[part][cluster] = list of docs numbe
partitions = []
for i in range(0, len(limits)):
    clusters = []
    for j in range(0, nb_clusters):
        clusters.append(get_doc(i,j))
    partitions.append(clusters)

In [18]:
partitions

[[[8,
   11,
   24,
   26,
   30,
   31,
   38,
   49,
   55,
   57,
   74,
   78,
   80,
   81,
   94,
   98,
   103,
   109,
   110,
   111],
  [0,
   3,
   4,
   10,
   12,
   22,
   27,
   41,
   42,
   44,
   51,
   53,
   56,
   58,
   65,
   75,
   85,
   92,
   95,
   108,
   113],
  [2,
   6,
   7,
   9,
   21,
   23,
   28,
   33,
   37,
   39,
   50,
   61,
   62,
   64,
   66,
   67,
   68,
   72,
   88,
   90,
   93,
   96,
   97,
   99,
   102,
   104,
   112],
  [15, 36, 63, 70, 79, 87],
  [1,
   5,
   13,
   14,
   16,
   17,
   18,
   19,
   20,
   25,
   29,
   32,
   34,
   35,
   40,
   43,
   45,
   46,
   47,
   48,
   52,
   54,
   59,
   60,
   69,
   71,
   73,
   76,
   77,
   82,
   83,
   84,
   86,
   89,
   91,
   100,
   101,
   105,
   106,
   107],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[114,
   120,
   121,
   123,
   132,
   137,
   140,
   149,

# Khi²

In [19]:
# tf_of_your_word = tf[numDoc][strWord]
tf = []
for doc in usable:
    tf_doc = {}
    for word in vectorizer.get_feature_names():
        tf_doc[word] = doc.count(word)
    tf.append(tf_doc)

In [20]:
# Number total of words
# nb_total_word[numPartition]
nb_total_word = []
nb = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        nb += tf[numDoc][word]
    if numDoc+1 in limits:
        nb_total_word.append(nb)
        nb=0
    

In [21]:
nb_total_word

[12667, 20096, 19095, 24614, 33564]

In [22]:
tf[0]

{'experimentalement': 0,
 'theorie': 0,
 'iteratif': 0,
 'tâche': 0,
 'attaquer': 0,
 'apprentissage supervis': 0,
 'collection document': 0,
 'metadonnee': 0,
 'lexploitation connaissance': 0,
 'papier': 0,
 'mis': 0,
 'classement': 0,
 'reference': 0,
 'dautre partir': 0,
 'limiter': 0,
 'cell': 0,
 'jeu': 0,
 'fouiller grand': 0,
 'cube': 0,
 'volum': 0,
 'introduire notion': 0,
 'lusage': 1,
 'latent': 0,
 'cluster': 0,
 'darbr': 0,
 'lespace representation': 0,
 'levaluation': 0,
 'extraction motif': 0,
 'fournir': 1,
 'montrer linteret': 0,
 'detr': 0,
 'mesurer similarite entrer': 0,
 'extraire connaissance': 0,
 'utilisateur': 0,
 'axiome': 0,
 'ensembliste': 0,
 'variabilite': 0,
 'methodologie': 0,
 'realisation': 0,
 'plaire pertinent': 0,
 'resumer': 0,
 'voir': 0,
 'dedonnee': 0,
 'ecd': 0,
 'dependanc': 0,
 'decrivons': 0,
 'lavantage': 0,
 'recent': 0,
 'annotation semantiqu': 0,
 'methode fouiller': 0,
 'ajoutee': 0,
 'complexite': 0,
 'porter': 0,
 'ontologie domaine':

In [23]:
# nb_word[num_partition][word]
nb_word = []

word_in_this_parti = {}
for word in vectorizer.get_feature_names():
    word_in_this_parti[word] = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        word_in_this_parti[word] += tf[numDoc][word]
    if numDoc+1 in limits:
        nb_word.append(word_in_this_parti)
        word_in_this_parti = {}
        for word in vectorizer.get_feature_names():
            word_in_this_parti[word] = 0

In [24]:
len(nb_word)

5

In [25]:
# nb_word_by_cluster[numPartition][numCluster]
nb_word_by_cluster = []
for parti in partitions:
    nb_word_clus = []
    for cluster in parti:
        nb = 0
        for numDoc in cluster:
            for word in vectorizer.get_feature_names():
                nb += tf[numDoc][word]
        nb_word_clus.append(nb)
    nb_word_by_cluster.append(nb_word_clus)

In [26]:
# value_of_khi2 = khi2[numPartition][numCluster][word]
khi2 = []

for numParti in range(0, len(partitions)):
    khi2parti = []
    for numCluster in range(0, len(partitions[numParti])):
        khi2cluster = {}
        
        for word in vectorizer.get_feature_names():
            if nb_word_by_cluster[numParti][numCluster] == 0:
                khi2cluster[word] = 0
            else:
                word_in_this_parti[word] = 0
                E = nb_word[numParti][word]
                E =+ nb_word_by_cluster[numParti][numCluster]
                E = E/ nb_total_word[numParti]
                N = 0
                for numDoc in partitions[numParti][numCluster]:
                    N += tf[numDoc][word]
                khi2cluster[word] = (pow(N - E, 2)/E)        
        khi2parti.append(khi2cluster)
    khi2.append(khi2parti)

In [27]:
# list of your labels = labels[numPartition][numCluster]
labels = []

for numPartition in range(0, len(nb_word_by_cluster)):
    label_clus = []
    for numCluster in range(0, len(nb_word_by_cluster[numPartition])):
        label_clus.append(Counter(khi2[numPartition][numCluster]).most_common(5))
    labels.append(label_clus)

In [48]:
# Some clusters can be empty
"donne" in labels[0][0]

False

# Diachronic Analysis

In [55]:
# TODO: sigma are ecart-type :)

def inter(listA, listB):
    return np.intersect1d(listA, listB)
    
# cluster_t and cluster_s must be in two different partitions
def proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S):
    total_inter = 0
    total_t = 0
    for f in range(0, len(labels[num_partition_T][num_cluster_t])):
        for f_s in labels[num_partition_S][num_cluster_s]:
            if labels[num_partition_T][num_cluster_t][f][0] == f_s[0]:
                total_inter += labels[num_partition_T][num_cluster_t][f][1]
                break
        total_t += labels[num_partition_T][num_cluster_t][f][1]
    if total_t == 0:
        return 0
    return total_inter / total_t
    

def P_A(num_cluster_s, num_partition_T, num_partition_S):
    # first, we have to know what are the cluster which got the label
    total = 0
    nb_computation = 0
    for label_s in labels[num_partition_S][num_cluster_s]:
        for num_cluster_t in range(0, len(partitions[num_partition_T])):
            if label_s in labels[num_partition_T][num_cluster_t]:
                total += proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S)
                nb_computation += 1
    if nb_computation == 0:
        return 0
    return total / nb_computation

# Define a coeficient for the activity 
def activity(num_partition_S, num_partition_T):
    res = 0
    for num_cluster_s in range(0, len(partitions[num_partition_S])):
        res += P_A(num_cluster_s, num_partition_T, num_partition_S)
    return res / len(partitions[num_partition_S])

# Ecart-type, but it isn't very usefull xD
sigma_t = 0.01
sigma_s = 0.01

# Our Graal
# Does cluster_t is similar to cluster_s?
def similar(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S):
    cond1 = proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S) > P_A(num_cluster_s, num_partition_T, num_partition_S)
    cond2 = proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S) > activity(num_partition_S, num_partition_T) + sigma_s
    
    cond1 = proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S) > P_A(num_cluster_s, num_partition_T, num_partition_S)
    cond2 = proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S) > activity(num_partition_T, num_partition_S) + sigma_t
    return cond1 and cond2 and cond3 and cond4
    

In [56]:
similar(3, 3, 0, 1)

False

In [57]:
print(labels[0][1])
labels[0][1][0][1]

[('donne', 4130.919130068158), ('utilis', 3839.2070727325768), ('donnee', 3287.8242734071123), ('text', 3028.1535314172306), ('method', 2779.1632278759153)]


4130.919130068158